# Facebook爬蟲
這裡簡單介紹一下爬蟲的邏輯再開始爬進入正題吧！

# 載入使用套件

In [8]:
import pandas as pd
import re, time, requests, datetime, gc, os
from selenium import webdriver
from bs4 import BeautifulSoup

# 經常用到的函數
Facebook請用戶「登入/註冊」的彈窗會反覆出現，在這裡先寫好嘗試點擊的函數提供使用

In [2]:
# 點擊不要現在註冊帳號
def ClickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(0.1)

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [3]:
# 截取塗鴉墻上貼文的留言數
def Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments

# 截取塗鴉墻上貼文的連結
def Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 
    for i in urls:
        driver.get(i)        
        for i in range(n):
            try:
                time.sleep(1)
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(1)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            ClickNotNow()
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':Wall_PostLink(i),
                                                                        'Time':Wall_PostTime(i),
                                                                        'Comments':Wall_PostComments(i)}],
                                                               columns = ['Link','Time','Comments'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [4]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    # ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    MoreComments()
    MoreReplies()
    SeeMore()

## 爬取貼文內容與留言資訊

In [5]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]

# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def Post_Shares(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def Post_Likes(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def Post_Loves(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def Post_Hahas(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def Post_Wows(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def Post_Sads(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def Post_Angrys(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Shares':Post_Shares(userContent),
                                        'Likes':Post_Likes(userContent),
                                        'Loves':Post_Loves(userContent),
                                        'Hahas':Post_Hahas(userContent),
                                        'Wows':Post_Wows(userContent),
                                        'Sads':Post_Sads(userContent),
                                        'Angrys':Post_Angrys(userContent),
                                        'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                        'Link':driver.current_url}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
    return PostContent

# 留言內容
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

def CommentsInfo(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':Comment_Content(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':Comment_Content(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [6]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [7]:
try:
    Posts = pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime'])

print(Posts.shape)
print(Comments.shape)
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

(1494, 14)
(222578, 8)


## 產出待爬清單

In [8]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [9]:
CarwlList = CarwlList(urls=urls, n=10, Posts = Posts)
CarwlList

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/tstartel/posts/332769...,2019-06-05 01:53,16,NaN,left_only
1,https://www.facebook.com/tstartel/photos/a.413...,2019-06-04 21:00,26,NaN,left_only
4,https://www.facebook.com/tstartel/posts/332506...,2019-06-04 03:31,32,NaN,left_only
5,https://www.facebook.com/tstartel/posts/332479...,2019-06-04 02:00,132,NaN,left_only
6,https://www.facebook.com/tstartel/photos/a.413...,2019-06-03 02:38,200,NaN,left_only
42,https://www.facebook.com/tstartel/photos/a.413...,2019-04-21 21:00,26,NaN,left_only
43,https://www.facebook.com/tstartel/photos/a.239...,2019-04-19 20:51,67,NaN,left_only
51,https://www.facebook.com/chtmobile/posts/27573...,2019-06-05 04:00,0,NaN,left_only
52,https://www.facebook.com/chtmobile/posts/27564...,2019-06-05 02:00,0,NaN,left_only
53,https://www.facebook.com/chtmobile/photos/a.26...,2019-06-04 02:00,7,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [10]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('Expand Succed!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            nPost = PostInfo(soup)
            nComments = CommentsInfo(soup)
            Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
            print('Update PostInfo complete!')
            Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
            print('Update CommentsInfo complete!')
        except:
            print('Update Post or CommentsInfo Failed!')
    except:
        print('This post can not expand')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/posts/3327691170590240
Click More Comments： 0 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-06 10:09:14

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3327111277314896/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-06 10:09:29

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3325060057520018
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-06 10:09:43

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3324790744213616
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More

This post can not expand
Time Log: 2019-06-06 10:12:58

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2480578298829263
This post can not expand
Time Log: 2019-06-06 10:13:02

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2476110232609403
This post can not expand
Time Log: 2019-06-06 10:13:05

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2476091885944571
This post can not expand
Time Log: 2019-06-06 10:13:08

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2474400126113747
This post can not expand
Time Log: 2019-06-06 10:13:12

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2470316226522137
This post can not expand
Time Log: 2019-06-06 10:13:15

------------------
Dealing with: https://www.facebook.com/taiwanmobile/posts/2469601513260275
This post can not expand
Time Log: 2019-06-06 10:13:19

------------------
Dealing with: https://www

Click More Replies： 23 times.
Click More Replies： 24 times.
Click More Replies： 25 times.
Click More Replies： 26 times.
Click More Replies： 27 times.
Click More Replies： 28 times.
Click More Replies： 29 times.
Click More Replies： 30 times.
Click More Replies： 31 times.
Click More Replies： 32 times.
Click More Replies： 33 times.
Click More Replies： 34 times.
Click More Replies： 35 times.
Click More Replies： 36 times.
Click More Replies： 37 times.
Click More Replies： 38 times.
Click More Replies： 39 times.
Click More Replies： 40 times.
Click More Replies： 41 times.
Click More Replies： 42 times.
Click More Replies： 43 times.
Click More Replies： 44 times.
Click More Replies： 45 times.
Click More Replies： 46 times.
Click More Replies： 47 times.
Click See More ： 1 times.
Click See More ： 2 times.
Click See More ： 3 times.
Click See More ： 4 times.
Click See More ： 5 times.
Click See More ： 6 times.
Click See More ： 7 times.
Click See More ： 8 times.
Click See More ： 9 times.
Click See More ：

Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-06 10:23:39

------------------
Dealing with: https://www.facebook.com/fareastone/posts/2563874770292476
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-06 10:24:17

------------------
Dealing with: https://www.facebook.com/fareastone/photos/a.163720023

In [11]:
driver.quit()

In [12]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,亞太電信Gt智慧生活,Aptg.tw,2019-05-29 02:08,【小編捕獲重量級人物👀 集團大家長郭總裁蒞臨指導】 台灣科技新創培育的最高殿堂的「2019 ...,9,9,118,1,0,0,0,2,2019-06-06 10:26,https://www.facebook.com/Aptg.tw/posts/2478267...
1,亞太電信Gt智慧生活,Aptg.tw,2019-05-27 20:00,義氣省錢再出擊‼ 亞太電信就是最挺你💪 超殺方案讓你省到不要不要der~ 🔥299不限速上網...,34,2,75,1,0,0,0,0,2019-06-06 10:26,https://www.facebook.com/Aptg.tw/photos/a.1653...
2,亞太電信Gt智慧生活,Aptg.tw,2019-06-03 20:08,0元❗0元❗ 沒錯～亞太就是要你0元帶回家❗ 省錢消暑大作戰!! 亞太電信推出多款熱賣手機超...,10,1,60,1,1,0,0,0,2019-06-06 10:25,https://www.facebook.com/Aptg.tw/photos/a.1653...
3,遠傳電信,fareastone,2019-05-15 20:50,【來遠傳買iPad 筆較划算】 熱騰騰的全新iPad Air、iPad mini開賣啦！ 全...,29,4,91,0,0,0,0,0,2019-06-06 10:25,https://www.facebook.com/fareastone/photos/a.1...
4,亞太電信Gt智慧生活,Aptg.tw,2019-06-03 00:00,Zenfone Max M2 超乎想像的「大」 追劇怕螢幕小😢出遊怕電不夠😭 Zenfone...,8,2,54,1,0,1,0,0,2019-06-06 10:25,https://www.facebook.com/Aptg.tw/photos/a.1653...
5,亞太電信Gt智慧生活,Aptg.tw,2019-06-02 20:00,股東獨享💝夏普家電特惠齊發第1彈💣 ㊗鴻海45週年㊗生日快樂🎉 亞太電信與夏普攜手聯歡 SH...,8,3,54,1,0,0,0,1,2019-06-06 10:25,https://www.facebook.com/Aptg.tw/photos/a.1653...
6,遠傳電信,fareastone,2019-05-25 20:45,#女孩兒來測你的生活哲學 小編今天突然想到，小時候常常幻想自己會變成怎樣的大人 當時間不知不...,239,5,101,0,0,0,0,0,2019-06-06 10:24,https://www.facebook.com/fareastone/photos/a.1...
7,遠傳電信,fareastone,2019-05-29 03:30,There's No Text Content!,297,12,131,0,0,0,0,0,2019-06-06 10:24,https://www.facebook.com/fareastone/photos/a.1...
8,遠傳電信,fareastone,2019-05-30 20:45,【外星人❓退差價❓ 到底誰說的是真的😵】 童顏童語好浮誇，揪~竟誰說的是真心話呢！ 遠傳...,973,948,1000,0,0,0,0,0,2019-06-06 10:24,https://www.facebook.com/fareastone/posts/2563...
9,遠傳電信,fareastone,2019-05-31 20:45,#夏季濕身秀 又來啦！霸託我不想流汗😖 最近雷雨+高溫，悶熱的天氣快受不了啦☀ 小編貴為...,27,16,52,0,0,0,0,0,2019-06-06 10:23,https://www.facebook.com/fareastone/posts/2563...


In [13]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100008036701674,陳國進,2019-05-29 17:27,哇~是郭台銘,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/posts/2478267...,2019-06-06 10:26
1,100002470664192,Chun Ju Wu,2019-05-28 21:15,"合約到繼續用吧，199=199,299=299",162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
2,162174107155573,亞太電信Gt智慧生活,2019-06-04 19:33,您好，若您現在的手機門號是屬於無合約的狀態，可以直接申辦188方案喔！,100024104980789,孫天龍,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
3,100000198195362,蔡文豪,2019-05-31 10:31,老客戶已沒有綁約，不能換約嗎？,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
4,100000173438302,余雅慧,2019-06-01 17:47,申辦期間到何時？,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
5,162174107155573,亞太電信Gt智慧生活,2019-06-03 13:37,您好，此方案的申辦期間到6/30喔方案說明請參考：https://reurl.cc/AbAn3,100000173438302,余雅慧,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
6,100000701465888,Jimo Luo,2019-05-31 07:55,有支援亞太全頻的手機。在我家很猛 4頻全有。 遠傳室內連3mb都沒 只有700頻段收的到。亞...,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
7,100000818895234,蘇文雅,2019-05-30 11:00,實體門市台南有哪家。 原是台哥大用戶 6月中合約才到期 這樣能申辦嗎,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
8,162174107155573,亞太電信Gt智慧生活,2019-05-30 12:42,您好，299方案目前只限定網路門市辦理哦~方案到5/31止，需提前與原電信解約後再進行申辦喔！,100000818895234,蘇文雅,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26
9,100000818895234,蘇文雅,2019-05-30 17:14,亞太電信Gt智慧生活 解約後不就不能用同一個號碼了,100000818895234,蘇文雅,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-06-06 10:26


# 保存資料

In [14]:
Posts.to_pickle('C:/Users/' + os.getlogin() + '/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/' + os.getlogin() + '/Desktop/Comments.plk')
Posts.to_csv('C:/Users/' + os.getlogin() + '/Desktop/Posts.csv')
Comments.to_csv('C:/Users/' + os.getlogin() + '/Desktop/Comments.csv')

# To Do List
- 故障(無法expand的貼文要避免加入貼文清單或者加入後，要讓下次再來爬文，不能直接跳過。)
 - 評估將加載更多留言放在同一個function中
 
 https://blog.csdn.net/qq_24499417/article/details/81408655

# 維護測試區

In [60]:
#driver = webdriver.Chrome()
#driver.get('https://www.facebook.com/')
#time.sleep(1)
#driver.find_element_by_partial_link_text('English').click()

In [11]:
nComments

NameError: name 'nComments' is not defined

In [61]:
#driver.get('https://www.facebook.com/tstartel/videos/1100239263492734/?permPage=1')
#time.sleep(0.7)
#PostExpand()

# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--disable-notifications")

#driver = webdriver.Chrome(options=chrome_options)
#url = 'https://www.facebook.com/'
#driver.get(url)
#time.sleep(3)

# 啟動瀏覽器並登入Facebook
#username = driver.find_element_by_id('email')
#username.send_keys('61034b001@gms.ndhu.edu.tw')
#passwd=driver.find_element_by_id('pass')
#passwd.send_keys('19920309')
#button=driver.find_element_by_id('loginbutton')
#button.click()